# Imports

In [2]:
# ! pip install pandas
# ! pip install calender
# ! pip install numpy
# ! pip install datetime
# ! pip install matplotlib
# ! pip install collections
# ! pip install random
# ! pip install tqdm
# ! pip install sklearn
# ! pip install lightgbm
# ! pip install xgboost

In [3]:
import pandas as pd
import calendar
from datetime import datetime
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from IPython.display import clear_output as cclear
from lightgbm import LGBMRegressor
import joblib

# Load data

In [4]:
def get_csv(X):
    return pd.read_csv(path+X)
path = ''
calender, sales_train_ev, sales_train_val, sell_prices = get_csv('calendar.csv'), get_csv('sales_train_evaluation.csv'), \
                                                         get_csv('sales_train_validation.csv'), get_csv('sell_prices.csv')

In [5]:
non_numeric_col_list = ['id','item_id','dept_id','cat_id','store_id','state_id','d', 'date']
store_dict = {'CA_1':0, 'CA_2':0, 'CA_3':0, 'CA_4':0, 'WI_1':0, 'WI_2':0, 'WI_3':0, 'TX_1':0, 'TX_2':0, 'TX_3':0}

In [30]:
# Encoding Categorical Columns
def encode_cat_cols(new_df):
    le = [0]*len(non_numeric_col_list)             
    for i in range(len(non_numeric_col_list)):
        print("Encoding col: ", non_numeric_col_list[i])
        le[i] = LabelEncoder()
        new_df[non_numeric_col_list[i]] = le[i].fit_transform( new_df[non_numeric_col_list[i]] )
    return le, new_df


# Function for reversing the long form
def reverse_long_form(le, X_test, train_out):
    for i in range(len(non_numeric_col_list)):
        X_test[non_numeric_col_list[i]] = le[i].inverse_transform(X_test[non_numeric_col_list[i]])

    X_test['unit_sale'] = train_out
    kk = X_test.pivot(index='id', columns='d')['unit_sale']
    kk['id'] = kk.index
    kk.reset_index(drop=True, inplace=True)

    cols = list(kk)
    cols = [cols[-1]] + cols[:-1]
    kk = kk[cols]

    return kk


# This function does feature engineering on sales_train_ev or sales_train_val
# There is another feature engineering function for adding columns to dataframe containing rows of only onw store
def feature_engineer(df):
    day_columns = list(df.columns[6:])
    other_var = list(df.columns[:6])
    
    print('Melting out...')
    df = pd.melt(df, id_vars = other_var, value_vars = day_columns)
    df = df.rename(columns = {"variable": "d", "value": "unit_sale"})
    # print(df.shape)
    
    print('Adding Feature \'date\'...')
    cal_dict = dict(zip(calender.d,calender.date))
    df["date"] = df["d"].map(cal_dict)
    # df.head()
    
    print('Adding Feature \'day_of_week\'...')
    day_of_week_dict = dict(zip(calender.d,calender.wday))
    df['day_of_week'] = df["d"].map(day_of_week_dict)

    print('Adding Feature \'month_no\'...')
    month_no_dict = dict(zip(calender.d,calender.month))
    df['month_no'] = df["d"].map(month_no_dict)
    
    print('Adding Feature \'day_of_month\'...')
    l = [i[-2:] for i in list(calender.date)]
    calender['day_of_month'] = l
    
    day_of_month_dict = dict(zip(calender.d,calender.day_of_month))
    df['day_of_month'] = df["d"].map(day_of_month_dict)
    
    print('Done.')
    print('Here is how featurised data looks like...')
    print(df.head(3))
    return df


def reorder_data(df, csv_name):
    df['sp_index'] = (df.index)
    index_dict = dict(zip(df.id, df.sp_index))
    df = df.drop('sp_index', axis=1)

    kk = pd.read_csv(str(csv_name)+'.csv')
    # kk = kk.drop(kk.columns[0], axis=1)

    kk['sp_index'] = kk["id"].map(index_dict)
    kk = kk.sort_values(by='sp_index', axis=0)
    kk = kk.drop('sp_index', axis=1)
    kk.to_csv(str(csv_name)+'.csv')

# Train Function

In [7]:
def startegy7dot1(new_df, dept):
    print('Using strategy ', strategy)
    evaluation, validation = new_df.id.iloc[0].find('evaluation'), new_df.id.iloc[0].find('validation')
    
    new_df = new_df[new_df.dept_id == dept]
    print('Total rows: ', len(new_df))
    
    rows_per_day = len(new_df[new_df.d == 'd_1'])
    print('Rows per day: ', rows_per_day)
    
    new_df['day_of_month'] = new_df['day_of_month'].fillna(0)
    new_df = new_df.astype({'day_of_month': 'int32'})                   # Making day_of_month column as int
    new_df['date'] = new_df['date'].astype(str)
    
    y = new_df.unit_sale                                                # getting the label
    new_df = new_df.drop('unit_sale', axis=1)

    print('Encoding categorical features...')
    le, new_df = encode_cat_cols(new_df)                                # Encoding Categorical Columns

    X = new_df
    
    ev_train_start, ev_train_end, val_train_start, val_train_end = rows_per_day*(0), rows_per_day*1941,\
                                                                   rows_per_day*(0), rows_per_day*1913
    
    model = LGBMRegressor(boosting_type = 'gbdt',       
                          objective = 'tweedie',
                          tweedie_variance_power = 1.3,    
                          metric = 'rmse',
                          subsample = 0.5,
                          subsample_freq = 1,
                          learning_rate = 0.03,
                          num_leaves = 3000,              
                          min_data_in_leaf = 5000,        
                          feature_fraction = 0.5,
                          max_bin = 300,                 
                          n_estimators = 500,
                          boost_from_average = False,
                          verbose = -1,
                          n_jobs = -1)
    
    if evaluation != -1:                                        # if evaluation data
        print('Getting X_train, y_train...')
        X_train, y_train = X.iloc[ev_train_start:ev_train_end], y[ev_train_start:ev_train_end] 
        X_test, y_test = X.iloc[ev_train_end:], y[ev_train_end:] 
        model_name = 'Eval_'+str(dept)+'.pkl'
        joblib.dump(le, 'le_Eval_'+str(dept)+'.pkl')            
        
    if validation != -1:                                        # if validation data
        print('Getting X_train, y_train...')
        X_train, y_train = X.iloc[val_train_start:val_train_end], y[val_train_start:val_train_end]
        X_test, y_test = X.iloc[val_train_end:], y[val_train_end:]
        model_name = 'Val_'+str(dept)+'.pkl'
        joblib.dump(le, 'le_Val_'+str(dept)+'.pkl')
    
    print('X_train len', len(X_train), 'y_train len', len(y_train), 'X_test len', len(X_test))
    
    print('Fitting model...')
    model.fit(X_train, y_train)
    
    print('Fitting done. Saving model...')
    joblib.dump(model, model_name)
    
    joblib_model = joblib.load(model_name)
    
    print('Making predictions...')
    train_out = joblib_model.predict(X_test)

    print('Done.')
    return le, X_test, train_out

def get_output_of_eval_or_val(df):
    main_out_df = pd.DataFrame()
    
    list_dept = list(set(df.dept_id))
    for i in list_dept:
        print('Sequence of depts processing: ', list_dept)
        print('Working on Dept: ', i)
        
        le, X_test, train_out = startegy7dot1(df, i)
        
        print('Reversing the long form...')
        out_df = reverse_long_form(le, X_test, train_out)
        main_out_df = pd.concat([main_out_df, out_df], ignore_index=False)
        cclear()

    l = []                                                       # In this part we rename the columns to F_1, F_2 ....
    for i in range(1,29):
        l.append('F'+str(i))
    l = ['id']+l

    main_out_df.columns = l
    
    return main_out_df

# Run

In [8]:
strategy = 7.1

In [9]:
############## Eval data

In [10]:
%%time
df = sales_train_ev.copy()
empty_list = [0]*30490
for i in range(1942, 1970):
    df['d_'+str(i)] = empty_list
df = feature_engineer(df)

Melting out...
Adding Feature 'date'...
Adding Feature 'day_of_week'...
Adding Feature 'month_no'...
Adding Feature 'day_of_month'...
Done.
Here is how featurised data looks like...
                              id        item_id    dept_id   cat_id store_id  \
0  HOBBIES_1_001_CA_1_evaluation  HOBBIES_1_001  HOBBIES_1  HOBBIES     CA_1   
1  HOBBIES_1_002_CA_1_evaluation  HOBBIES_1_002  HOBBIES_1  HOBBIES     CA_1   
2  HOBBIES_1_003_CA_1_evaluation  HOBBIES_1_003  HOBBIES_1  HOBBIES     CA_1   

  state_id    d  unit_sale        date  day_of_week  month_no day_of_month  
0       CA  d_1          0  2011-01-29            1         1           29  
1       CA  d_1          0  2011-01-29            1         1           29  
2       CA  d_1          0  2011-01-29            1         1           29  
CPU times: user 32.6 s, sys: 8.82 s, total: 41.4 s
Wall time: 41.3 s


In [11]:
%%time
main_out_df_ev = get_output_of_eval_or_val(df)
main_out_df_ev.to_csv('main_out_ev.csv', index=False)

CPU times: user 4h 35min 40s, sys: 1min 14s, total: 4h 36min 54s
Wall time: 51min 14s


In [12]:
############# Val Data

In [13]:
%%time
df = sales_train_val.copy()
empty_list = [0]*30490
for i in range(1914, 1942):
    df['d_'+str(i)] = empty_list
df = feature_engineer(df)

Melting out...
Adding Feature 'date'...
Adding Feature 'day_of_week'...
Adding Feature 'month_no'...
Adding Feature 'day_of_month'...
Done.
Here is how featurised data looks like...
                              id        item_id    dept_id   cat_id store_id  \
0  HOBBIES_1_001_CA_1_validation  HOBBIES_1_001  HOBBIES_1  HOBBIES     CA_1   
1  HOBBIES_1_002_CA_1_validation  HOBBIES_1_002  HOBBIES_1  HOBBIES     CA_1   
2  HOBBIES_1_003_CA_1_validation  HOBBIES_1_003  HOBBIES_1  HOBBIES     CA_1   

  state_id    d  unit_sale        date  day_of_week  month_no day_of_month  
0       CA  d_1          0  2011-01-29            1         1           29  
1       CA  d_1          0  2011-01-29            1         1           29  
2       CA  d_1          0  2011-01-29            1         1           29  
CPU times: user 38 s, sys: 9.85 s, total: 47.9 s
Wall time: 47.7 s


In [14]:
%%time
main_out_df_val = get_output_of_eval_or_val(df)
main_out_df_val.to_csv('main_out_val.csv', index=False)

CPU times: user 4h 15min 3s, sys: 1min 8s, total: 4h 16min 12s
Wall time: 47min 47s


In [15]:
############# Reorder and Write the output

In [17]:
sales_train_val

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913,sp_index
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,3,0,1,1,1,3,0,1,1,0
1,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,0,0,1,0,0,0,0,1
2,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,2,1,1,1,0,1,1,1,2
3,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,5,4,1,0,1,3,7,2,3
4,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,1,0,1,1,2,2,2,4,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30485,FOODS_3_823_WI_3_validation,FOODS_3_823,FOODS_3,FOODS,WI_3,WI,0,0,2,2,...,0,0,0,0,0,1,0,0,1,30485
30486,FOODS_3_824_WI_3_validation,FOODS_3_824,FOODS_3,FOODS,WI_3,WI,0,0,0,0,...,0,0,0,0,0,0,0,1,0,30486
30487,FOODS_3_825_WI_3_validation,FOODS_3_825,FOODS_3,FOODS,WI_3,WI,0,6,0,2,...,1,0,2,0,1,0,0,1,0,30487
30488,FOODS_3_826_WI_3_validation,FOODS_3_826,FOODS_3,FOODS,WI_3,WI,0,0,0,0,...,0,1,0,0,1,0,3,1,3,30488


In [31]:
reorder_data(sales_train_val, 'main_out_val')
reorder_data(sales_train_ev, 'main_out_ev')

In [32]:
main_out_ev = pd.read_csv('main_out_ev.csv')
main_out_val = pd.read_csv('main_out_val.csv')

sub_df = pd.concat([main_out_ev, main_out_val], ignore_index=True)
sub_df = sub_df.round(2)

In [36]:
sub_df.drop([sub_df.columns[0], sub_df.columns[-1]], axis=1, inplace=True)

In [37]:
sub_df

,id,F1,F2,F3,F4,F5,F6,F7,F8,F9,...,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
0,HOBBIES_1_001_CA_1_evaluation,0.69,0.58,0.57,0.56,0.56,0.73,0.70,0.57,0.51,...,0.55,0.74,0.71,0.56,0.49,0.49,0.50,0.57,0.78,0.76
1,HOBBIES_1_002_CA_1_evaluation,0.58,0.49,0.48,0.47,0.47,0.63,0.60,0.49,0.43,...,0.47,0.64,0.61,0.48,0.42,0.42,0.43,0.48,0.66,0.65
2,HOBBIES_1_003_CA_1_evaluation,1.01,0.85,0.85,0.81,0.84,1.26,1.20,0.91,0.80,...,0.84,1.28,1.22,0.89,0.77,0.77,0.79,0.85,1.31,1.26
3,HOBBIES_1_004_CA_1_evaluation,1.01,0.85,0.85,0.81,0.84,1.26,1.20,0.91,0.80,...,0.84,1.28,1.22,0.89,0.77,0.77,0.79,0.85,1.31,1.26
4,HOBBIES_1_005_CA_1_evaluation,1.31,1.17,1.18,1.12,1.28,1.84,1.68,1.28,1.17,...,1.33,1.91,1.77,1.29,1.16,1.17,1.17,1.33,1.94,1.81
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60975,FOODS_3_823_WI_3_validation,0.44,0.39,0.38,0.38,0.48,0.63,0.57,0.51,0.49,...,0.48,0.69,0.72,0.49,0.43,0.41,0.41,0.47,0.63,0.63
60976,FOODS_3_824_WI_3_validation,0.34,0.30,0.30,0.30,0.37,0.49,0.44,0.39,0.38,...,0.37,0.53,0.55,0.38,0.33,0.32,0.32,0.37,0.48,0.49
60977,FOODS_3_825_WI_3_validation,0.93,0.84,0.82,0.82,1.00,1.13,1.01,1.04,1.03,...,0.98,1.21,1.26,1.00,0.91,0.89,0.88,0.99,1.13,1.13
60978,FOODS_3_826_WI_3_validation,1.15,1.07,1.04,1.04,1.24,1.35,1.19,1.26,1.28,...,1.19,1.42,1.49,1.22,1.15,1.14,1.13,1.23,1.35,1.35


In [38]:
sub_df.to_csv('submission.csv', index=False)